In [83]:
def retrieve_review():
    # Example for MySQL
    connection = pymysql.connect(
        host='localhost',
        user='root',
        password='1234',
        database='shoutreview',
        autocommit=True
    )
    
    cursor = connection.cursor()
    # Query to extract reviews
    query = "select r.id, r.review from review r where r.sentiment is NULL"
    cursor.execute(query)
    reviews = cursor.fetchall()
    
    # Convert the reviews from tuple format
    reviews = [review for review in reviews]
    
    # Close connection
    # connection.close()
    return reviews
retrieve_review()

[]

In [84]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def preprocess_review(text):
    # Lowercasing
    text = text.lower()
    # Remove special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenization
    tokens = text.split()
    # Remove stop words
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    # Join tokens back to string
    return ' '.join(tokens)

def get_cleaned_reviews(reviews):
    cleaned_reviews = [preprocess_review(review[1]) for review in reviews]
    return reviews, cleaned_reviews
get_cleaned_reviews(retrieve_review())

[nltk_data] Downloading package stopwords to C:\Users\Kriti
[nltk_data]     Srivastava\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


([], [])

In [85]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

def get_sentiment(cleaned_reviews):
    sia = SentimentIntensityAnalyzer()
    
    # Analyze sentiment
    sentiments = [sia.polarity_scores(review) for review in get_cleaned_reviews(retrieve_review())[1]]
    
    # Print first review and its sentiment
    #print(sentiments[0])
    reviews= [review[0] for review in get_cleaned_reviews(retrieve_review())[0]]
    return sentiments, reviews
get_sentiment(get_cleaned_reviews(retrieve_review())[1])

[nltk_data] Downloading package vader_lexicon to C:\Users\Kriti
[nltk_data]     Srivastava\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


([], [])

In [86]:
def classify_sentiment(score):
    if score > 0:
        return 'positive'
    elif score == 0:
        return 'neutral'
    else:
        return 'negative'

In [87]:
from flask import Flask, request, jsonify
import pymysql

# Initialize Flask app
app = Flask(__name__)


@app.route('/analyze', methods=['POST'])
def analyze_sentiment():  
    try:
        connection = pymysql.connect(
        host='localhost',
        user='root',
        password='1234',
        database='shoutreview',
        autocommit=True)
        cursor = connection.cursor()
        
        review= retrieve_review()
        review= get_cleaned_reviews(review)
        sentiments, review= get_sentiment(review)
        sentiment_labels = [classify_sentiment(sentiment['compound']) for sentiment in sentiments]

        for sentiment, review_id in zip(sentiment_labels, review):
            insert_query = "UPDATE review SET sentiment = %s WHERE id = %s"
            cursor.execute(insert_query, (sentiment, review_id))
        connection.commit()
    except Exception as e:
        return jsonify({'error': str(e)}), 500
    finally:
        connection.close()
    
    # Return the sentiment analysis result
    return jsonify({
        'message' : 'Done',
        'sentiment' : sentiment_labels[0]
    })


# Run the app
if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Oct/2024 22:57:53] "POST /analyze HTTP/1.1" 200 -
